In [ ]:
import os
from improc.io import parse_collection, DCAccessor
DCAccessor.register()

import numpy as np
import holoviews as hv
hv.extension('bokeh')

# parsing a data collection into an indexable dataframe
The parsing pattern follows python string formatting style. e.g. parsing time on 4 digits: `'{time:04d}'`

In [ ]:
datadir = '../../data/2D'
data_pattern = '{platedir}/{layer}/{f1}_{f2}_{f3}_{row}{col:02d}_T{T:04d}F{field:03d}L{L:02d}A{A:02d}Z{zslice:02d}C{channel:02d}.{ext}'
index = ['platedir', 'layer', 'row', 'col', 'field', 'zslice', 'channel']

df = parse_collection(os.path.join(datadir,data_pattern), index)

df

# build selection menu from a data collection

CollectionHandler builds an interactive menu matching the index/multi-index of a dataframe. The indexed dataframe is available as "subdf" attribute.

In [ ]:
from inter_view.io import CollectionHandler

collection_handler = CollectionHandler(df=df)
collection_handler.panel()

# extension allowing multiple selection on chosen index levels
can for instance be used to interactively chelect channels

In [ ]:
from inter_view.io import MultiCollectionHandler

collection_handler = MultiCollectionHandler(df=df, multi_select_levels=['zslice', 'channel'])
collection_handler.panel()

# automatically loading files associated with current sub-collection

DataLoader extends MultiCollectionHandler. It expects the dataframe to corresponds to files available on disc and indexable/readable with `dc` pandas accessor (see data collection parsing step). Every time the sub-collection changes, corresponding files are automatically loaded and available under dictionary argument `loaded_objects`.

Reading functions available trough pandas `dc` accessor are used by default. A custom files reading function `loading_fun` can also be provided. e.g. to only read the first channel of multi-channel files. If multiple files are selected, `loading_fun` reads files in parallel with multi-threading.

In [ ]:
from inter_view.io import DataLoader
from skimage.io import imread

def read_first_slice(path):
    return imread(path, img_num=0)

data_loader = DataLoader(df=df,
                         multi_select_levels=['channel'],
                         loading_fun=read_first_slice,)
data_loader.panel()

In [ ]:
data_loader.loaded_objects